# Probabilistic Time Series Analysis

## Week 10: Spectral Gaussian Processes Methods

Places where you are supposed to fill in code are marked

    #
    # TODO: some instructions
    # 
    
The rest of the code we will run and discuss if time permits, otherwise try it out at home and try to answer the questions mentioned in the text boxes for yourself.

You will need to install a new package:

    pyGPs

### Please turn in the code before 12/12/2018 5:20pm. 

### Your work will be evaluated based on the code and plots. You don't need to write down your answers to other questions in the text blocks, just think them over.

### Title your submission file `lab11-student-[YOUR NET ID].ipynb`.

# Setup

In [ ]:
import numpy as np
import pandas as pd
import pyGPs
import GPy
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
import scipy.spatial.distance as spdist

# Patch of initSMhypers(self, x, y) in pyGPs
def initSMhypers(model, x, y):
    x = np.atleast_2d(x)
    y = np.atleast_2d(y)
    (n, D) = x.shape
    Q = model.covfunc.para[0]
    w = np.zeros(Q)
    m = np.zeros((D, Q))
    s = np.zeros((D, Q))
    w[:] = np.std(y) / Q
    hypinit = np.zeros(Q + 2 * D * Q)

    for i in range(D):
        # Calculate distances
        xslice = np.atleast_2d(x[:, i]).T
        d2 = spdist.cdist(xslice, xslice, 'sqeuclidean')
        if n > 1:
            d2[d2 == 0] = d2[0, 1]
        else:
            d2[d2 == 0] = 1
        minshift = np.min(np.min(np.sqrt(d2)))
        nyquist = 0.5 / minshift
        m[i, :] = nyquist * np.random.ranf((1, Q))
        maxshift = np.max(np.max(np.sqrt(d2)))
        s[i, :] = 1. / np.abs(maxshift * np.random.ranf((1, Q)))
    hypinit[:Q] = np.log(w)
    hypinit[Q + np.arange(0, Q * D)] = np.log(m[0, :]).T
    hypinit[Q + Q * D + np.arange(0, Q * D)] = np.log(s[0, :]).T
    model.covfunc.hyp = list(hypinit)

In [ ]:
def layer_plot(x, mu, std):
    plt.plot(x, mu, c='#003300', label='Mean')
    plt.plot(x, mu + std, c='#339933', label='One Standard Deviation')
    plt.plot(x, mu - std, c='#339933')
    plt.plot(x, mu + 2 * std, c='#9fdf9f', label='Two Standard Deviations')
    plt.plot(x, mu - 2 * std, c='#9fdf9f')
    plt.fill_between(x, mu, mu + std, color='#d9f2d9')
    plt.fill_between(x, mu, mu - std, color='#d9f2d9')
    plt.fill_between(x, mu + std, mu + 2 * std, color='#ecf9ec')
    plt.fill_between(x, mu - std, mu - 2 * std, color='#ecf9ec')

# A Look at the Data

In this assignment, you will recreate Figure 1 of the paper "Gaussian Process Kernels for Pattern Discovery and Extrapolation" [Wilson, Adams] where spectral mixture kernels where introduced. This uses data measuring CO2 concentration collected at an observatory in Hawaii. Let's take a look at the full dataset.

In [ ]:
co2_df = pd.read_csv('../../data/co2.csv', delim_whitespace=True, comment='#', header=None)
# Truncate to avoid some misread points; this will start around 1985:
X_CO2 = co2_df[2][314:]
Y_CO2 = co2_df[3][314:]

In [ ]:
plt.plot(X_CO2, Y_CO2)

We will divide this into training and test data, cutting off at 2010.

In [ ]:
X_CO2_train = X_CO2[X_CO2 <= 2010]
X_CO2_test = X_CO2[X_CO2 > 2010]

num_train_points = X_CO2_train.shape[0]
num_test_points = X_CO2_test.shape[0]

Y_CO2_train = Y_CO2[:num_train_points]
Y_CO2_test = Y_CO2[num_train_points:]

X_CO2_train = X_CO2_train.reshape((num_train_points, 1))
X_CO2_test = X_CO2_test.reshape((num_test_points, 1))
Y_CO2_train = Y_CO2_train.reshape((num_train_points, 1))
Y_CO2_test = Y_CO2_test.reshape((num_test_points, 1))

# Using a Simple Model

In [ ]:
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m_reg = GPy.models.GPRegression(X_CO2_train, Y_CO2_train, kernel)
_ = m_reg.optimize()
_ = m_reg.plot(plot_density=True)

In [ ]:
mean, var = m_reg.predict(X_CO2_test)
layer_plot(X_CO2_test[:, 0], mean[:, 0], np.sqrt(var)[:, 0])
plt.plot(X_CO2_test, Y_CO2_test, c='black', label='True Values', linewidth=3)
plt.legend()

Terrible!

# Using a Spectral Mixture Model

Here is an example of how we run prediction with a spectral mixture model. The parameter `Q` controls how many components of the gaussian mixture are allowed.

In [ ]:
Q = 3

In [ ]:
kernel = pyGPs.cov.SM(Q=Q, D=1)

model = pyGPs.GPR()
model.setData(X_CO2_train, Y_CO2_train)
model.setPrior(mean=pyGPs.mean.Zero(), kernel=kernel)
initSMhypers(model, X_CO2_train, Y_CO2_train)

model.setOptimizer('Minimize', num_restarts=10)
model.getPosterior()
model.optimize()

In [ ]:
mean, var, _, _, _ = model.predict(X_CO2_test)
layer_plot(X_CO2_test[:, 0], mean[:, 0], np.sqrt(var[:, 0]))
plt.plot(X_CO2_test, Y_CO2_test, c='black', label='True Values', linewidth=3)
plt.legend()

Still terrible!

In [ ]:
#
# TODO: Experiment with the parameter Q until you get good results above. 
# What is the smallest value giving good prediction? Compare with what [Adams, Wilson] report.
#

# Examining the Spectral Density

Let's extract the parameters that were learned. This determines a mixture of gaussians: a collection of weights, proportional to which we choose a gaussian with given means and variances.

In [ ]:
weights = np.exp(model.covfunc.hyp[:Q])
spectral_means = np.exp(np.reshape(model.covfunc.hyp[Q:Q + Q * 1], (1, Q)))
spectral_variances = np.exp(2 * np.reshape(model.covfunc.hyp[Q + Q * 1:], (1, Q)))

In [ ]:
#
# TODO: By sampling from the gaussian mixture model, draw a histogram of the spectral density.
# If necessary, draw one histogram for the most likely component, and a separate one for the others (if one
# component is too likely, the others will not get picked often enough for you to estimate well). If necessary,
# plot on a log scale to make the differences clear. Compare with Figure 1(b) of [Adams, Wilson].
#